In [6]:
import os
# to use or not to use GPU
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

# packages for model graph visualization
#!pip install -q pydot
# install graphviz https://graphviz.gitlab.io/download/ 

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub
!pip install -q -U tensorflow-text
import tensorflow_text as text
!pip install -q tf-models-official
from official.nlp import optimization  # to create AdamW optimizer

import json
import re
import random
import numpy as np
import matplotlib.pyplot as plt

#save_path = "/content/drive/My Drive/Colab Notebooks/"

     |████████████████████████████████| 4.9 MB 8.7 MB/s 
     |████████████████████████████████| 1.8 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 90 kB 11.5 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 1.1 MB 53.8 MB/s 
     |████████████████████████████████| 99 kB 12.4 MB/s 
     |████████████████████████████████| 1.2 MB 53.2 MB/s 
     |████████████████████████████████| 352 kB 72.7 MB/s 
     |████████████████████████████████| 213 kB 76.1 MB/s 


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
save_path = "/content/drive/My Drive/Colab Notebooks/"

In [9]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [10]:
# checking if we have access to a GPU

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
#@title Data Loading & Preprocessing


In [12]:
with open(save_path+'project_data_regex.json') as f:
    data = json.load(f)
    # shuffle data
    random.shuffle(data)
    # create description and assignee list
    desc_data = []
    assignee_data = []
    for item in data:
        desc_data.append(item['description'])
        assignee_data.append(item['assignee'])

In [13]:
print('sample assignees:', assignee_data[0:2])
print('sample descriptions:', desc_data[0:2])

sample assignees: ['Jun Rao', 'Ewen Cheslack-Postava']
sample descriptions: ['Broker shuts down with the following error message 013 01 11 01 43 51 320 ERROR KafkaApis request expiration task kafka KafkaApi 277 error when processing request service metrics 2 39192 2000000 java nio channels ClosedChannelException at sun nio ch FileChannelImpl ensureOpen FileChannelImpl java 88 at sun nio ch FileChannelImpl read FileChannelImpl java 613 at kafka log FileMessageSet searchFor FileMessageSet scala 82 at kafka log LogSegment translateOffset LogSegment scala 76 at kafka log LogSegment read LogSegment scala 106 at kafka log Log read Log scala 386 at kafka server KafkaApis kafka server KafkaApis readMessageSet KafkaApis scala 369 at kafka server KafkaApis anonfun kafka server KafkaApis readMessageSets 1 apply KafkaApis scala 327 at kafka server KafkaApis anonfun kafka server KafkaApis readMessageSets 1 apply KafkaApis scala 323 at scala collection TraversableLike anonfun map 1 apply Traversable

In [14]:
# transform description data to tensorflow dataset
descriptions = tf.data.Dataset.from_tensor_slices(desc_data)
for input in descriptions.take(2):
    print(input)

tf.Tensor(b'Broker shuts down with the following error message 013 01 11 01 43 51 320 ERROR KafkaApis request expiration task kafka KafkaApi 277 error when processing request service metrics 2 39192 2000000 java nio channels ClosedChannelException at sun nio ch FileChannelImpl ensureOpen FileChannelImpl java 88 at sun nio ch FileChannelImpl read FileChannelImpl java 613 at kafka log FileMessageSet searchFor FileMessageSet scala 82 at kafka log LogSegment translateOffset LogSegment scala 76 at kafka log LogSegment read LogSegment scala 106 at kafka log Log read Log scala 386 at kafka server KafkaApis kafka server KafkaApis readMessageSet KafkaApis scala 369 at kafka server KafkaApis anonfun kafka server KafkaApis readMessageSets 1 apply KafkaApis scala 327 at kafka server KafkaApis anonfun kafka server KafkaApis readMessageSets 1 apply KafkaApis scala 323 at scala collection TraversableLike anonfun map 1 apply TraversableLike scala 206 at scala collection TraversableLike anonfun map 1 a

In [29]:
# target vectorize assignee data
assignee_dict = {assignee: i for i, assignee in enumerate(list(set(assignee_data)))}

In [19]:
# create target vector
assignee_vector = [assignee_dict[assignee] for assignee in assignee_data]
assignee_vector[:10]

[13, 17, 6, 11, 23, 20, 6, 10, 21, 34]

In [26]:
# assignee target vector to one hot vector
assignee_tensor = tf.data.Dataset.from_tensor_slices(assignee_vector)

for assignee in assignee_tensor.take(2):
    print(assignee)

tf.Tensor(13, shape=(), dtype=int32)
tf.Tensor(17, shape=(), dtype=int32)


In [ ]:
#@title BOW


In [30]:
input_vectorizer = layers.TextVectorization(
    split = "whitespace",
    output_mode='multi_hot')

In [31]:
input_vectorizer.adapt(descriptions)

In [32]:
output_vectorizer = layers.CategoryEncoding(num_tokens = len(assignee_dict), output_mode="one_hot")

In [34]:
for description in descriptions.take(2):
    print(description)
    print(input_vectorizer(description))

tf.Tensor(b'Broker shuts down with the following error message 013 01 11 01 43 51 320 ERROR KafkaApis request expiration task kafka KafkaApi 277 error when processing request service metrics 2 39192 2000000 java nio channels ClosedChannelException at sun nio ch FileChannelImpl ensureOpen FileChannelImpl java 88 at sun nio ch FileChannelImpl read FileChannelImpl java 613 at kafka log FileMessageSet searchFor FileMessageSet scala 82 at kafka log LogSegment translateOffset LogSegment scala 76 at kafka log LogSegment read LogSegment scala 106 at kafka log Log read Log scala 386 at kafka server KafkaApis kafka server KafkaApis readMessageSet KafkaApis scala 369 at kafka server KafkaApis anonfun kafka server KafkaApis readMessageSets 1 apply KafkaApis scala 327 at kafka server KafkaApis anonfun kafka server KafkaApis readMessageSets 1 apply KafkaApis scala 323 at scala collection TraversableLike anonfun map 1 apply TraversableLike scala 206 at scala collection TraversableLike anonfun map 1 a

In [33]:
for bow_assignee in assignee_tensor.take(2):
    print(bow_assignee)
    print(output_vectorizer(bow_assignee))

tf.Tensor(13, shape=(), dtype=int32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor(17, shape=(), dtype=int32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)


In [35]:
bow_dataset = tf.data.Dataset.zip((descriptions, assignee_tensor))

def vectorize_dataset(description, assignee):

  return input_vectorizer(description), output_vectorizer(assignee)

bow_dataset = bow_dataset.map(vectorize_dataset)

for description, assignee in bow_dataset.take(2):
    print(description, assignee)

tf.Tensor([0. 1. 1. ... 0. 0. 0.], shape=(23690,), dtype=float32) tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor([0. 1. 0. ... 0. 0. 0.], shape=(23690,), dtype=float32) tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)


In [36]:
bow_dataset = bow_dataset.shuffle(len(list(bow_dataset)), seed = 42)

In [37]:
# batching the dataset
BATCH_SIZE = 32
VAL_SIZE = 0.15
bow_dataset = bow_dataset.batch(batch_size=BATCH_SIZE)

for line, label in bow_dataset.take(2):
    print(line.shape)
    
print(len(list(bow_dataset)))

(32, 23690)
(32, 23690)
136


In [38]:
# splitting the dataset into validation and test data
val_size = round(VAL_SIZE*(len(list(bow_dataset))))

val_dataset = bow_dataset.take(val_size) 
train_dataset = bow_dataset.skip(val_size)

In [40]:
len(list(train_dataset))

116

In [41]:
vocab_size = input_vectorizer.vocabulary_size()
bow_model = tf.keras.Sequential([
  tf.keras.layers.Dense(64, input_shape=(vocab_size,), activation='relu'),
  tf.keras.layers.Dense(len(assignee_dict), activation = "softmax")
])

In [42]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [43]:
bow_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [44]:
bow_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1516224   
                                                                 
 dense_1 (Dense)             (None, 36)                2340      
                                                                 
Total params: 1,518,564
Trainable params: 1,518,564
Non-trainable params: 0
_________________________________________________________________


In [46]:
for inputs,targets in train_dataset.take(1):
    print(inputs)
    outputs = bow_model(inputs)

    print(outputs.shape)
    print(outputs)
    print(targets)

tf.Tensor(
[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]], shape=(32, 23690), dtype=float32)
(32, 36)
tf.Tensor(
[[0.03192618 0.02916428 0.02592525 ... 0.02684351 0.02909265 0.02769976]
 [0.02831022 0.02670729 0.02759043 ... 0.02805972 0.02920359 0.02837124]
 [0.02662924 0.02757234 0.02635361 ... 0.02726456 0.02962496 0.02886213]
 ...
 [0.02882663 0.02768573 0.02806281 ... 0.0278467  0.02833084 0.03160632]
 [0.02948579 0.02795951 0.02695216 ... 0.02777952 0.0294066  0.0281682 ]
 [0.02931223 0.02606308 0.02829448 ... 0.02742473 0.03223411 0.03087921]], shape=(32, 36), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]], shape=(32, 36), dtype=float32)


In [47]:
bow_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


116/116 [==============================] - 4s 18ms/step - loss: 3.3055 - accuracy: 0.1574 - val_loss: 2.6699 - val_accuracy: 0.4047
Epoch 2/15
116/116 [==============================] - 3s 17ms/step - loss: 2.3987 - accuracy: 0.4611 - val_loss: 1.7989 - val_accuracy: 0.6844
Epoch 3/15
116/116 [==============================] - 3s 17ms/step - loss: 1.5922 - accuracy: 0.7061 - val_loss: 1.1442 - val_accuracy: 0.8406
Epoch 4/15
116/116 [==============================] - 3s 17ms/step - loss: 1.0223 - accuracy: 0.8542 - val_loss: 0.7189 - val_accuracy: 0.9125
Epoch 5/15
116/116 [==============================] - 3s 17ms/step - loss: 0.6817 - accuracy: 0.9194 - val_loss: 0.4685 - val_accuracy: 0.9703
Epoch 6/15
116/116 [==============================] - 3s 17ms/step - loss: 0.4513 - accuracy: 0.9584 - val_loss: 0.2995 - val_accuracy: 0.9766
Epoch 7/15
116/116 [==============================] - 3s 17ms/step - loss: 0.3121 - accuracy: 0.9767 - val_loss: 0.2283 - val_accuracy: 0.9875
Epoch 8/15